In [1]:
# Make jupyter include changes to source code, without having to restart.
%load_ext autoreload
%autoreload 2

In [9]:
import h5py # For some reason, needs to be imported first

import os
import sys
from pathlib import Path
from typing import List

from dolfin import *
from dolfin.function.argument import Argument
import numpy as np
import matplotlib.pyplot as plt

from neuralthreesome.solver_knpemi import Solver as SolverKNPEMI
from neuralthreesome.meshprocessing import hdf2fenics, geo2hdf
from neuralthreesome.subdomains import SubMeshCollection, FacetDomainInfo, SubDomainInfo
from neuralthreesome.ion import Ion, get_default_ion_list
from neuralthreesome.parameters import get_default_parameters
from neuralthreesome.solver_synapse import SynapseSolver, unpack_function_space

## General Setup

In [1]:
# Path to any results.
resultspath = Path('../results/')
resultspath.mkdir(parents=True, exist_ok=True)

# Path to gmsh-file, and where to output the resulting meshfile
geofile = "../resources/synapse-simple.geo"
meshfile = "../mesh/synapse-simple.h5"

# Create mesh from the gmsh-file
geo2hdf(geofile, meshfile, cleanup=False)

# Load the mesh and the MeshFunctions tagging the different subdomains.
mesh, subdomain_tags, boundary_tags = hdf2fenics(meshfile)


# Information of subdomains.
subdomain_info = [
    SubDomainInfo(
        name="exterior",
        value=1,
        interfaces=["post-membrane", "post-terminal"],
        ext_bdrys=['empty', 'pre', 'astrocytes']
    ),
    SubDomainInfo(
        name="interior",
        value=2,
        interfaces=["post-membrane", "post-terminal"],
        ext_bdrys=["post-boundary"]
    )
]

# Add information about boundaries and interfaces.
boundary_info = [
    FacetDomainInfo("pre", 1),
    FacetDomainInfo("astrocytes", 2),
    FacetDomainInfo("post-membrane", 3, bordering_regions=("interior", "exterior")),
    FacetDomainInfo("post-terminal", 4, bordering_regions=("interior", "exterior")),
    FacetDomainInfo("post-boundary", 5),
    FacetDomainInfo("empty", 6)
]

# Create a collection of submeshes for the subdomains and boundaries.
domain = SubMeshCollection(mesh, subdomain_tags, boundary_tags, subdomain_info, boundary_info)

# Write the facet MeshFunctionSizet from each submesh to a paraview-inspectable 
# file to verify correctness.
test_vtk = File(str(resultspath / "test_boundaries.pvd"))
for subdomain in domain.subdomains:
    test_vtk << subdomain.boundary_tags

# How to access mesh of a specific subdomain
plot(domain.subdomains[0])
plt.show()

NameError: name 'Path' is not defined

In [11]:
def set_ion_conductivities(ions: List[float], params):
    for ion in ions:
        ion.g_k = params["g_{}_leak".format(ion.name)]
    return ions

def add_ion_conductivity(ions: List[float], ion_name: str, conductivity: float):
    for ion in ions:
        if ion.name == ion_name:
            ion.g_k += conductivity

In [43]:
# time variables (seconds)
dt = 0.2e-5                 # global time step (s)
Tstop = 1.0e-4                  # global end time (s)

params = get_default_parameters()
ion_list = get_default_ion_list(params)

# synaptic current
g_syn_C1 = Expression(
    'g_syn_bar*(x[0] <= 40e-6)',
    g_syn_bar=params['g_syn_bar'],
    degree=4
)

t_1a = Constant(0.0)                                        # time constant? IS this word correct?
fname_1a = resultspath / "test"

set_ion_conductivities(ion_list, params)
add_ion_conductivity(ion_list, "Na", g_syn_C1)


# solve system
solver = SynapseSolver(domain, ion_list, t_1a, lagrange_tag="exterior", **params)  # create solver

In [44]:
solver.solve_system_passive(dt, Tstop, filename=str(resultspath))           # solve

100%|██████████| 50/50 [00:04<00:00, 10.15it/s]


# GRAVEYARD

In [13]:
def check_missing_blocks(solver):
    alist = extract_blocks(solver.a)
    Llist = extract_blocks(solver.L)
    for idx, ai in enumerate(alist):
        print(idx, ai is None)

solver = SynapseSolver(domain, ion_list, Constant(0.0), lagrange_tag="exterior", **params)
solver.create_variational_form(domain, "exterior")

# Define Test and TrialFunctions
u, I = unpack_function_space(TrialFunctions(solver.W), solver)
v, q = unpack_function_space(TestFunctions(solver.W), solver)

alist = extract_blocks(solver.a)
Llist = extract_blocks(solver.L)

# A = MixedLinearVariationalProblem(alist, Llist, solver.wh.split(), [])

iface = domain.interfaces[0]

# for iface in domain.interfaces:
check_missing_blocks(solver)

alist = extract_blocks(solver.a)
Llist = extract_blocks(solver.L)

# MixedLinearVariationalProblem(alist, Llist, solver.wh.split(), [])
print(" ==== Assemble L ===== ")
for i in range(4):
    try: 
        print(assemble_mixed(Llist[i]))
    except ValueError as e:
        print(e)

print()
print(" ==== Assemble A ===== ")

for i in range(16):
    print(i, end=" ")
    try:
        print(assemble_mixed(alist[i]))
    except (ValueError, AttributeError) as e:
        print(e)

assemble_mixed_system(solver.a == solver.L, solver.wh, [])

TypeError: unsupported operand type(s) for /: 'float' and 'SubMeshCollection'